### 5. Валидация и обработка ошибок

Валидация данных — это важная часть работы с формами в Django, так как она обеспечивает корректность данных, поступающих от пользователя. Django предоставляет встроенные механизмы валидации, а также даёт возможность настраивать собственные правила валидации для полей формы. Ошибки, которые возникают в процессе валидации, можно легко отображать в шаблонах, предоставляя пользователю понятную обратную связь.

#### 5.1 Встроенная валидация в Django

Django автоматически проводит валидацию данных формы на основе типов полей, используемых в форме или модели. Например:
- Поля типа `CharField` автоматически проверяются на наличие и длину текста.
- Поля типа `EmailField` проверяют формат введённого email-адреса.
- Поля типа `IntegerField` проверяют, что введённое значение является числом.

##### Пример встроенной валидации:

```python
from django import forms

class SimpleForm(forms.Form):
    name = forms.CharField(max_length=100)
    age = forms.IntegerField(min_value=18)

# В этом примере:
# - Поле name проверяется на то, чтобы длина не превышала 100 символов.
# - Поле age проверяется на то, что введённое значение — целое число не менее 18.
```

Когда форма будет отправлена, Django автоматически выполнит эти проверки. Если данные не проходят валидацию, ошибки будут добавлены в объект формы.

##### Использование метода `is_valid()` для запуска валидации

Метод `is_valid()` проверяет данные формы и возвращает `True`, если все данные прошли валидацию, и `False`, если есть ошибки. После вызова метода `is_valid()`, все ошибки становятся доступны через атрибут `form.errors`.

##### Пример использования `is_valid()`:

```python
def process_form(request):
    if request.method == 'POST':
        form = SimpleForm(request.POST)
        if form.is_valid():
            # Данные прошли валидацию
            name = form.cleaned_data['name']
            age = form.cleaned_data['age']
        else:
            # Ошибки валидации
            errors = form.errors
```


#### 5.2 Переопределение методов `clean()` и `clean_<field>()` для кастомной валидации

Когда встроенной валидации недостаточно для ваших нужд, вы можете легко добавить свою собственную логику валидации с помощью методов `clean()` и `clean_<field>()`.

- Метод `clean_<field>()` используется для валидации конкретного поля.
- Метод `clean()` используется для валидации всей формы в целом, когда требуется проверять зависимость значений нескольких полей друг от друга.

##### Переопределение метода `clean_<field>()`

Метод `clean_<field>()` позволяет добавить логику валидации для конкретного поля. Этот метод должен возвращать очищенные данные или вызывать исключение `ValidationError`, если данные невалидны.

##### Пример:

```python
from django import forms
from django.core.exceptions import ValidationError

class SimpleForm(forms.Form):
    name = forms.CharField(max_length=100)
    age = forms.IntegerField(min_value=18)

    def clean_age(self):
        age = self.cleaned_data.get('age')
        if age < 21:
            raise ValidationError('Возраст должен быть не менее 21 года.')
        return age
```

В этом примере поле `age` сначала проходит стандартную проверку на минимальное значение 18, но затем в методе `clean_age()` добавляется дополнительное условие: возраст должен быть не меньше 21 года. Если условие не выполнено, выбрасывается ошибка валидации.

##### Переопределение метода `clean()` для всей формы

Метод `clean()` используется для валидации нескольких полей одновременно или для выполнения комплексных проверок, например, когда одно поле зависит от другого.

##### Пример:

```python
class RegistrationForm(forms.Form):
    username = forms.CharField(max_length=100)
    password = forms.CharField(widget=forms.PasswordInput)
    confirm_password = forms.CharField(widget=forms.PasswordInput)

    def clean(self):
        cleaned_data = super().clean()
        password = cleaned_data.get('password')
        confirm_password = cleaned_data.get('confirm_password')

        if password and confirm_password and password != confirm_password:
            raise forms.ValidationError('Пароли не совпадают.')
```

Здесь метод `clean()` проверяет, что поля `password` и `confirm_password` совпадают. Если они не равны, выбрасывается ошибка.

Важно отметить, что метод `clean()` должен вызывать `super().clean()` для доступа к уже очищенным данным и обеспечения работы базовой валидации.


#### 5.3 Обработка ошибок в шаблонах

Когда форма не проходит валидацию, ошибки сохраняются в атрибуте `errors` объекта формы. Эти ошибки можно легко отобразить в шаблонах Django. Ошибки могут быть показаны для отдельных полей, а также общие ошибки формы.

##### Пример отображения ошибок в шаблоне:

1. В представлении создайте форму и обработайте возможные ошибки:

```python
def registration_view(request):
    if request.method == 'POST':
        form = RegistrationForm(request.POST)
        if form.is_valid():
            # Обработка валидной формы
            return redirect('success')
    else:
        form = RegistrationForm()
    return render(request, 'registration.html', {'form': form})
```

2. В шаблоне **`registration.html`** добавьте код для вывода ошибок:

```html
<form method="post">
    {% csrf_token %}

    <!-- Общие ошибки формы -->
    {% if form.non_field_errors %}
        <div class="alert alert-danger">
            {{ form.non_field_errors }}
        </div>
    {% endif %}

    <!-- Ошибки конкретных полей -->
    <div>
        {{ form.username.label_tag }}<br>
        {{ form.username }}<br>
        {% if form.username.errors %}
            <ul class="errorlist">
                {% for error in form.username.errors %}
                    <li>{{ error }}</li>
                {% endfor %}
            </ul>
        {% endif %}
    </div>

    <div>
        {{ form.password.label_tag }}<br>
        {{ form.password }}<br>
        {% if form.password.errors %}
            <ul class="errorlist">
                {% for error in form.password.errors %}
                    <li>{{ error }}</li>
                {% endfor %}
            </ul>
        {% endif %}
    </div>

    <div>
        {{ form.confirm_password.label_tag }}<br>
        {{ form.confirm_password }}<br>
        {% if form.confirm_password.errors %}
            <ul class="errorlist">
                {% for error in form.confirm_password.errors %}
                    <li>{{ error }}</li>
                {% endfor %}
            </ul>
        {% endif %}
    </div>

    <button type="submit">Регистрация</button>
</form>
```

В этом примере:
- `form.non_field_errors` выводит общие ошибки, которые не привязаны к конкретным полям (например, ошибка проверки совпадения паролей).
- Для каждого поля формы (`username`, `password`, `confirm_password`) выводятся ошибки, если они есть, в виде списка `<ul class="errorlist">`.

Вывод ошибок с помощью `form.<field>.errors` автоматически добавляет сообщения об ошибках, которые были сгенерированы в процессе валидации, будь то встроенная валидация или кастомная.

Django предоставляет встроенную валидацию форм и удобные методы для обработки ошибок. Вы можете использовать стандартные механизмы валидации или добавить свою логику валидации с помощью методов `clean()` и `clean_<field>()`. Ошибки легко отображаются в шаблонах, что помогает пользователям понимать, какие данные были введены некорректно и как их исправить.